# Import Library

In [ ]:
!pip install Sastrawi
!pip install swifter

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import numpy as np
import re
import string
import gspread
import seaborn as sns
from nltk.tokenize import word_tokenize 
from nltk.probability import FreqDist
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import swifter

In [ ]:
# import nltk
# nltk.download('all')

In [ ]:
# from google.colab import auth
# auth.authenticate_user()
# from google.auth import default

# creds, _ = default()
# gc = gspread.authorize(creds)

# worksheet = gc.open('twitter_lebaran.csv').sheet1
# rows = worksheet.get_all_values()
# df = pd.DataFrame.from_records(rows)

In [ ]:
df = pd.read_csv('/content/twitter_lebaran.csv')
df_label = pd.read_excel('/content/dataset.xlsx')
# stop_word = 

# Overview Data

In [ ]:
df.head()

,Unnamed: 0,text,tanggal,bahasa
0,0,Hey @txtdrbogor coba himbau warganya cek serti...,2022-05-07 15:57:35+00:00,in
1,1,RT @Outstandjing: Mumpung netizen progresif da...,2022-05-07 15:37:26+00:00,in
2,2,@octavianusMP Selamat malam Kak. Tidak ada per...,2022-05-07 15:29:56+00:00,in
3,3,Tambahan info yg musti dipersiapkan:\n~ paspor...,2022-05-07 15:25:01+00:00,in
4,4,"Tahan zra, lo udah vaksin booster",2022-05-07 15:23:46+00:00,tr


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2368 entries, 0 to 2367
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  2368 non-null   int64 
 1   text        2368 non-null   object
 2   tanggal     2368 non-null   object
 3   bahasa      2368 non-null   object
dtypes: int64(1), object(3)
memory usage: 74.1+ KB


In [ ]:
df_label.head()

,label,text
0,netral,hey coba himbau warganya cek seifikat vaksin d...
1,netral,mumpung netizen progresif dan berwawasan diban...
2,negative,ini setelah vaksin booster jadi batuk mana awe...
3,positive,maju loe bible gak takut gw gw dah vaksin booster
4,positive,buat sarapan besok sebelum vaksin booster


In [ ]:
df_label.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 255 entries, 0 to 254
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   label   254 non-null    object
 1   text    254 non-null    object
dtypes: object(2)
memory usage: 4.1+ KB


# Preprocessing

In [ ]:
# df.drop(0, axis = 0, inplace = True)
# df.keys()
df = df.drop(['Unnamed: 0'], axis=1)

In [ ]:
# df_label.drop('id', axis = 1, inplace = True)

In [ ]:
# df.rename(columns={1: 'text', 2: 'tanggal', 3: 'bahasa'}, inplace=True)

In [ ]:
df.head()

,text,tanggal,bahasa
0,Hey @txtdrbogor coba himbau warganya cek serti...,2022-05-07 15:57:35+00:00,in
1,RT @Outstandjing: Mumpung netizen progresif da...,2022-05-07 15:37:26+00:00,in
2,@octavianusMP Selamat malam Kak. Tidak ada per...,2022-05-07 15:29:56+00:00,in
3,Tambahan info yg musti dipersiapkan:\n~ paspor...,2022-05-07 15:25:01+00:00,in
4,"Tahan zra, lo udah vaksin booster",2022-05-07 15:23:46+00:00,tr


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2368 entries, 0 to 2367
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   text     2368 non-null   object
 1   tanggal  2368 non-null   object
 2   bahasa   2368 non-null   object
dtypes: object(3)
memory usage: 55.6+ KB


In [ ]:
df_label.head()

,label,text
0,netral,hey coba himbau warganya cek seifikat vaksin d...
1,netral,mumpung netizen progresif dan berwawasan diban...
2,negative,ini setelah vaksin booster jadi batuk mana awe...
3,positive,maju loe bible gak takut gw gw dah vaksin booster
4,positive,buat sarapan besok sebelum vaksin booster


In [ ]:
df_label.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 255 entries, 0 to 254
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   label   254 non-null    object
 1   text    254 non-null    object
dtypes: object(2)
memory usage: 4.1+ KB


In [ ]:
df_label.dropna(subset = ["label"], axis = 0, inplace = True)
df_label.dropna(subset = ["text"], axis = 0, inplace = True)

In [ ]:
df_label.drop(df_label[(df_label['label'] == '-')].index, inplace=True)
df_label.drop(df_label[(df_label['label'] == '?')].index, inplace=True)

In [ ]:
df = df.drop_duplicates()

In [ ]:
df_label['label'].value_counts()

netral      94
positive    81
negative    79
Name: label, dtype: int64

### Datetime to Date

In [ ]:
df['tanggal'] = pd.to_datetime(df["tanggal"]).dt.date

In [ ]:
df['tanggal'].unique()

array([datetime.date(2022, 5, 7), datetime.date(2022, 5, 6),
       datetime.date(2022, 5, 5), datetime.date(2022, 5, 4),
       datetime.date(2022, 5, 3), datetime.date(2022, 5, 2)], dtype=object)

In [ ]:
df = df.drop(['tanggal'], axis=1)

### Drop bahasa != 'in'

In [ ]:
df = df.drop(df[(df['bahasa'] != 'in')].index)

In [ ]:
df = df.drop(['bahasa'], axis=1)

## text

In [ ]:
df['text_real'] = df.text

### Mengubah huruf menjadi kecil

In [ ]:
df["text"] = df["text"].apply(lambda item: str(item).lower())
df_label["text"] = df_label["text"].apply(lambda item: str(item).lower())

### Menghapus spesial karakter dan url

In [ ]:
def remove_text_special(text):
    # hapus tab, new line, ans back slice
    text = text.replace('\\t'," ").replace('\\n'," ").replace('\\u'," ").replace('\\',"")
    # hapus non ASCII (emot, bahasa china dll)
    text = text.encode('ascii', 'replace').decode('ascii')
    # hapus mention, link, hashtag
    text = text.replace("rt", "") # revisi
    text = ' '.join(re.sub("([@#][A-Za-z0-9]+)|(\w+:\/\/\S+)"," ", text).split())
    # remove URL 
    return text.replace("http://", " ").replace("https://", " ")

In [ ]:
df['text'] = df.text.astype('str')                
df['text'] = df['text'].apply(remove_text_special)
df_label['text'] = df_label.text.astype('str')                
df_label['text'] = df_label['text'].apply(remove_text_special)

### Menghapus tanda baca

In [ ]:
def remove_punctuation(text):
    return text.translate(str.maketrans("","",string.punctuation))

In [ ]:
df['text'] = df['text'].apply(remove_punctuation)
df_label['text'] = df_label['text'].apply(remove_punctuation)

### Menghapus enter tab

In [ ]:
def remove_whitespace_LT(text):
    return text.strip()

In [ ]:
df['text'] = df['text'].apply(remove_whitespace_LT)
df_label['text'] = df_label['text'].apply(remove_whitespace_LT)

### Menghapus spasi ganda

In [ ]:
def remove_whitespace_multiple(text):
    return re.sub('\s+',' ',text)

In [ ]:
df['text'] = df['text'].apply(remove_whitespace_multiple)
df_label['text'] = df_label['text'].apply(remove_whitespace_multiple)

### Menghapus kata berulang

In [ ]:
#remove kata yang sengaja dipanjangkan
# seperti bisaaaaaaa -> bisa
def spell(word):
    if re.search(r'([a-zA-Z])\1{2,}', word):
        rword = re.sub(r'([a-zA-Z])\1{2,}','\\1', word)
        return rword
    else:
        return word

In [ ]:
df['text'] = df['text'].apply(spell)
df_label['text'] = df_label['text'].apply(spell)

In [ ]:
print(df.iloc[0]["text"])

hey coba himbau warganya cek seifikat vaksin di peduli lindungi kalian belum vaksin booster sudah ada seifikat nya


In [ ]:
print(df_label.iloc[100]["text"])

wisnu yantie nakula den walaupun sudah vaksin booster jangan lupa tetap disiplin protokol kesehatan demi keselamatan kita semua


In [ ]:
df.head()

,text,text_real
0,hey coba himbau warganya cek seifikat vaksin d...,Hey @txtdrbogor coba himbau warganya cek serti...
1,mumpung netizen progresif dan berwawasan diban...,RT @Outstandjing: Mumpung netizen progresif da...
2,selamat malam kak tidak ada persyaratan khusus...,@octavianusMP Selamat malam Kak. Tidak ada per...
3,tambahan info yg musti dipersiapkan paspor kl ...,Tambahan info yg musti dipersiapkan:\n~ paspor...
5,ini syarat utbk diugm pake vaksin boosterswab ...,ini syarat utbk diugm pake vaksin booster/swab...


In [ ]:
df_label.tail()

,label,text
250,positive,jd ak abis vaksin booster jam set 10 an td trs...
251,netral,btw gue percaya banget sih kalau efeknya dahsy...
252,positive,alhamdulillah saya sudah disuntik booster sehi...
253,negative,booster side effect lagi teruk dari vaksin seb...
254,negative,di negri wakanda nun jauh disana udh gak gini ...


## export excel

In [ ]:
df_excel = pd.DataFrame(df.text, columns=['text']) 

In [ ]:
df_excel.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2288 entries, 0 to 2367
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    2288 non-null   object
dtypes: object(1)
memory usage: 35.8+ KB


In [ ]:
df_excel = df_excel.drop_duplicates()
df['text'] = df['text'].drop_duplicates()

In [ ]:
df.dropna(axis = 0, inplace = True)

In [ ]:
df_excel.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1520 entries, 0 to 2349
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    1520 non-null   object
dtypes: object(1)
memory usage: 23.8+ KB


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1520 entries, 0 to 2349
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   text       1520 non-null   object
 1   text_real  1520 non-null   object
dtypes: object(2)
memory usage: 35.6+ KB


In [ ]:
# file_name = "twitter_lebaran_bersih.xlsx"

# df_excel.to_excel(file_name)

## tokenize

In [ ]:
from nltk.tokenize import word_tokenize 

In [ ]:
def word_tokenize_wrapper(text):
    return word_tokenize(text)
    
def tokenize(word):
  word = word.split(" ")
  return word

In [ ]:
df_train = pd.merge(df, df_label, on='text', how='inner')

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
df_train['text_tokens'] = df_train['text'].apply(word_tokenize_wrapper)
# df_train['text_tokens'] = df_train['text'].apply(tokenize)

In [ ]:
df_train['label'].value_counts()

positive    87
netral      42
negative    36
Name: label, dtype: int64

In [ ]:
df_train.head()

,text,text_real,label,text_tokens
0,hey coba himbau warganya cek seifikat vaksin d...,Hey @txtdrbogor coba himbau warganya cek serti...,netral,"[hey, coba, himbau, warganya, cek, seifikat, v..."
1,mumpung netizen progresif dan berwawasan diban...,RT @Outstandjing: Mumpung netizen progresif da...,netral,"[mumpung, netizen, progresif, dan, berwawasan,..."
2,ini setelah vaksin booster jadi batuk mana awe...,Ini setelah vaksin booster jadi batukkkk.. man...,negative,"[ini, setelah, vaksin, booster, jadi, batuk, m..."
3,maju loe bible gak takut gw gw dah vaksin booster,"RT @rockmansick: MAJU LOE BIBLE GAK TAKUT GW, ...",positive,"[maju, loe, bible, gak, takut, gw, gw, dah, va..."
4,maju loe bible gak takut gw gw dah vaksin booster,"MAJU LOE BIBLE GAK TAKUT GW, GW DAH VAKSIN BOO...",positive,"[maju, loe, bible, gak, takut, gw, gw, dah, va..."


## count word

In [ ]:
from collections import defaultdict
word_count = defaultdict(int)
for tokens in df_train["text_tokens"]:
    for token in tokens:
       word_count[token] += 1

In [ ]:
word_count_df = pd.DataFrame({"key": word_count.keys(), "count": word_count.values()})
word_count_df_sort = word_count_df.sort_values('count', axis=0, ascending=False, inplace=False)
word_count_df_sort.head()

,key,count
6,vaksin,207
12,booster,178
13,sudah,45
350,untuk,44
86,jangan,39


## Menghapus stopwords

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
from nltk.corpus import stopwords

In [ ]:
list_stopwords = stopwords.words('indonesian')
list_stopwords.extend(["yg", "dg", "rt", "dgn", "ny", "d", 'klo', 
                       'kalo', 'amp', 'biar', 'bikin', 'bilang', 
                       'gak', 'ga', 'krn', 'nya', 'nih', 'sih', 
                       'si', 'tau', 'tdk', 'tuh', 'utk', 'ya', 
                       'jd', 'jgn', 'sdh', 'aja', 'n', 't', 
                       'nyg', 'hehe', 'pen', 'u', 'nan', 'loh', 'rt',
                       '&amp', 'yah'])

txt_stopword = pd.read_csv("/content/stopwords.txt", names= ["stopwords"], header = None)
list_stopwords.extend(txt_stopword["stopwords"][0].split(' '))
list_stopwords = set(list_stopwords)
def stopwords_removal(words):
    return [word for word in words if word not in list_stopwords]
df_train['text_tokens_WSW'] = df_train['text_tokens'].apply(stopwords_removal) 

In [ ]:
df_train.head()

,text,text_real,label,text_tokens,text_tokens_WSW
0,hey coba himbau warganya cek seifikat vaksin d...,Hey @txtdrbogor coba himbau warganya cek serti...,netral,"[hey, coba, himbau, warganya, cek, seifikat, v...","[coba, himbau, warganya, cek, seifikat, vaksin..."
1,mumpung netizen progresif dan berwawasan diban...,RT @Outstandjing: Mumpung netizen progresif da...,netral,"[mumpung, netizen, progresif, dan, berwawasan,...","[mumpung, netizen, progresif, berwawasan, diba..."
2,ini setelah vaksin booster jadi batuk mana awe...,Ini setelah vaksin booster jadi batukkkk.. man...,negative,"[ini, setelah, vaksin, booster, jadi, batuk, m...","[vaksin, booster, batuk, awet, sampe, skrng, y..."
3,maju loe bible gak takut gw gw dah vaksin booster,"RT @rockmansick: MAJU LOE BIBLE GAK TAKUT GW, ...",positive,"[maju, loe, bible, gak, takut, gw, gw, dah, va...","[maju, bible, takut, vaksin, booster]"
4,maju loe bible gak takut gw gw dah vaksin booster,"MAJU LOE BIBLE GAK TAKUT GW, GW DAH VAKSIN BOO...",positive,"[maju, loe, bible, gak, takut, gw, gw, dah, va...","[maju, bible, takut, vaksin, booster]"


## Distribusi frekuensi NLTK

In [ ]:
def freqDist_wrapper(text):
    return FreqDist(text)

df_train['text_tokens_fdist'] = df_train['text_tokens'].apply(freqDist_wrapper)

print('Frequency Tokens : \n') 
print(df_train['text_tokens_fdist'].head().apply(lambda x : x.most_common()))

Frequency Tokens : 

0    [(seifikat, 2), (vaksin, 2), (hey, 1), (coba, ...
1    [(udah, 2), (mumpung, 1), (netizen, 1), (progr...
2    [(ini, 1), (setelah, 1), (vaksin, 1), (booster...
3    [(gw, 2), (maju, 1), (loe, 1), (bible, 1), (ga...
4    [(gw, 2), (maju, 1), (loe, 1), (bible, 1), (ga...
Name: text_tokens_fdist, dtype: object


## Filtering (penghapusan stopwords) / normalisasi

In [ ]:
from nltk.corpus import stopwords
list_stopwords = stopwords.words('indonesian')

# lanjutan


In [ ]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import swifter

In [ ]:
df_train.head()

,text,text_real,label,text_tokens,text_tokens_WSW,text_tokens_fdist
0,hey coba himbau warganya cek seifikat vaksin d...,Hey @txtdrbogor coba himbau warganya cek serti...,netral,"[hey, coba, himbau, warganya, cek, seifikat, v...","[coba, himbau, warganya, cek, seifikat, vaksin...","{'hey': 1, 'coba': 1, 'himbau': 1, 'warganya':..."
1,mumpung netizen progresif dan berwawasan diban...,RT @Outstandjing: Mumpung netizen progresif da...,netral,"[mumpung, netizen, progresif, dan, berwawasan,...","[mumpung, netizen, progresif, berwawasan, diba...","{'mumpung': 1, 'netizen': 1, 'progresif': 1, '..."
2,ini setelah vaksin booster jadi batuk mana awe...,Ini setelah vaksin booster jadi batukkkk.. man...,negative,"[ini, setelah, vaksin, booster, jadi, batuk, m...","[vaksin, booster, batuk, awet, sampe, skrng, y...","{'ini': 1, 'setelah': 1, 'vaksin': 1, 'booster..."
3,maju loe bible gak takut gw gw dah vaksin booster,"RT @rockmansick: MAJU LOE BIBLE GAK TAKUT GW, ...",positive,"[maju, loe, bible, gak, takut, gw, gw, dah, va...","[maju, bible, takut, vaksin, booster]","{'maju': 1, 'loe': 1, 'bible': 1, 'gak': 1, 't..."
4,maju loe bible gak takut gw gw dah vaksin booster,"MAJU LOE BIBLE GAK TAKUT GW, GW DAH VAKSIN BOO...",positive,"[maju, loe, bible, gak, takut, gw, gw, dah, va...","[maju, bible, takut, vaksin, booster]","{'maju': 1, 'loe': 1, 'bible': 1, 'gak': 1, 't..."


In [ ]:
df_normalized = pd.DataFrame(df_train['label'])
df_normalized['text'] = df_train['text']

## menjadikan kata baku

In [ ]:
normalizad_word = df_normalized

normalizad_word_dict = {}

for index, row in normalizad_word.iterrows():
    if row[0] not in normalizad_word_dict:
        normalizad_word_dict[row[0]] = row[1] 

def normalized_term(document):
    return [normalizad_word_dict[term] if term in normalizad_word_dict else term for term in document]

df_train['text_normalized'] = df_train['text_tokens_WSW'].apply(normalized_term)


# create stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# stemmed
def stemmed_wrapper(term):
    return stemmer.stem(term)

term_dict = {}

for document in df_train['text_normalized']:
    for term in document:
        if term not in term_dict:
            term_dict[term] = ' '
            
print(len(term_dict))
print("------------------------")

for term in term_dict:
    term_dict[term] = stemmed_wrapper(term)
    print(term,":" ,term_dict[term])
    
print(term_dict)
print("------------------------")


# apply stemmed term to dataframe
def get_stemmed_term(document):
    return [term_dict[term] for term in document]

df_train['text_tokens_stemmed'] = df_train['text_normalized'].swifter.apply(get_stemmed_term)

483
------------------------
coba : coba
himbau : himbau
warganya : warga
cek : cek
seifikat : seifikat
vaksin : vaksin
peduli : peduli
lindungi : lindung
booster : booster
mumpung : mumpung
netizen : netizen
progresif : progresif
berwawasan : wawas
dibanding : banding
keluarga : keluarga
kampung : kampung
bahas : bahas
1 : 1
2 : 2
batuk : batuk
awet : awet
sampe : sampe
skrng : skrng
yaallah : yaallah
cape : cape
maju : maju
bible : bible
takut : takut
sarapan : sarap
besok : besok
makannya : makan
dijamin : jamin
kemana : mana
ajah : ajah
bebas : bebas
mudahmudahan : mudahmudahan
mudik : mudik
berangkat : berangkat
suntik : suntik
terinfeksi : infeksi
aktif : aktif
berfungsi : fungsi
infeksi : infeksi
virus : virus
covid : covid
19 : 19
tunggu : tunggu
kabar : kabar
satgas : satgas
kelonggaran : longgar
pakai : pakai
surat : surat
mewajibkan : wajib
seolaholah : seolaholah
pemer : per
kle : kle
info : info
denpasar : denpasar
kuta : kuta
ton : ton
ajak : ajak
jalan2 : jalan2
pake : p

Pandas Apply:   0%|          | 0/165 [00:00<?, ?it/s]

In [ ]:
df_train.head()

,text,text_real,label,text_tokens,text_tokens_WSW,text_tokens_fdist,text_normalized,text_tokens_stemmed
0,hey coba himbau warganya cek seifikat vaksin d...,Hey @txtdrbogor coba himbau warganya cek serti...,netral,"[hey, coba, himbau, warganya, cek, seifikat, v...","[coba, himbau, warganya, cek, seifikat, vaksin...","{'hey': 1, 'coba': 1, 'himbau': 1, 'warganya':...","[coba, himbau, warganya, cek, seifikat, vaksin...","[coba, himbau, warga, cek, seifikat, vaksin, p..."
1,mumpung netizen progresif dan berwawasan diban...,RT @Outstandjing: Mumpung netizen progresif da...,netral,"[mumpung, netizen, progresif, dan, berwawasan,...","[mumpung, netizen, progresif, berwawasan, diba...","{'mumpung': 1, 'netizen': 1, 'progresif': 1, '...","[mumpung, netizen, progresif, berwawasan, diba...","[mumpung, netizen, progresif, wawas, banding, ..."
2,ini setelah vaksin booster jadi batuk mana awe...,Ini setelah vaksin booster jadi batukkkk.. man...,negative,"[ini, setelah, vaksin, booster, jadi, batuk, m...","[vaksin, booster, batuk, awet, sampe, skrng, y...","{'ini': 1, 'setelah': 1, 'vaksin': 1, 'booster...","[vaksin, booster, batuk, awet, sampe, skrng, y...","[vaksin, booster, batuk, awet, sampe, skrng, y..."
3,maju loe bible gak takut gw gw dah vaksin booster,"RT @rockmansick: MAJU LOE BIBLE GAK TAKUT GW, ...",positive,"[maju, loe, bible, gak, takut, gw, gw, dah, va...","[maju, bible, takut, vaksin, booster]","{'maju': 1, 'loe': 1, 'bible': 1, 'gak': 1, 't...","[maju, bible, takut, vaksin, booster]","[maju, bible, takut, vaksin, booster]"
4,maju loe bible gak takut gw gw dah vaksin booster,"MAJU LOE BIBLE GAK TAKUT GW, GW DAH VAKSIN BOO...",positive,"[maju, loe, bible, gak, takut, gw, gw, dah, va...","[maju, bible, takut, vaksin, booster]","{'maju': 1, 'loe': 1, 'bible': 1, 'gak': 1, 't...","[maju, bible, takut, vaksin, booster]","[maju, bible, takut, vaksin, booster]"


## Menampilkan kembali gabungan data

In [ ]:
def merge_again(text):
    return ' '.join(text)

In [ ]:
df_train['done_text'] = df_train['text_tokens_stemmed'].apply(merge_again)

In [ ]:
df_train.head()

,text,text_real,label,text_tokens,text_tokens_WSW,text_tokens_fdist,text_normalized,text_tokens_stemmed,done_text
0,hey coba himbau warganya cek seifikat vaksin d...,Hey @txtdrbogor coba himbau warganya cek serti...,netral,"[hey, coba, himbau, warganya, cek, seifikat, v...","[coba, himbau, warganya, cek, seifikat, vaksin...","{'hey': 1, 'coba': 1, 'himbau': 1, 'warganya':...","[coba, himbau, warganya, cek, seifikat, vaksin...","[coba, himbau, warga, cek, seifikat, vaksin, p...",coba himbau warga cek seifikat vaksin peduli l...
1,mumpung netizen progresif dan berwawasan diban...,RT @Outstandjing: Mumpung netizen progresif da...,netral,"[mumpung, netizen, progresif, dan, berwawasan,...","[mumpung, netizen, progresif, berwawasan, diba...","{'mumpung': 1, 'netizen': 1, 'progresif': 1, '...","[mumpung, netizen, progresif, berwawasan, diba...","[mumpung, netizen, progresif, wawas, banding, ...",mumpung netizen progresif wawas banding keluar...
2,ini setelah vaksin booster jadi batuk mana awe...,Ini setelah vaksin booster jadi batukkkk.. man...,negative,"[ini, setelah, vaksin, booster, jadi, batuk, m...","[vaksin, booster, batuk, awet, sampe, skrng, y...","{'ini': 1, 'setelah': 1, 'vaksin': 1, 'booster...","[vaksin, booster, batuk, awet, sampe, skrng, y...","[vaksin, booster, batuk, awet, sampe, skrng, y...",vaksin booster batuk awet sampe skrng yaallah ...
3,maju loe bible gak takut gw gw dah vaksin booster,"RT @rockmansick: MAJU LOE BIBLE GAK TAKUT GW, ...",positive,"[maju, loe, bible, gak, takut, gw, gw, dah, va...","[maju, bible, takut, vaksin, booster]","{'maju': 1, 'loe': 1, 'bible': 1, 'gak': 1, 't...","[maju, bible, takut, vaksin, booster]","[maju, bible, takut, vaksin, booster]",maju bible takut vaksin booster
4,maju loe bible gak takut gw gw dah vaksin booster,"MAJU LOE BIBLE GAK TAKUT GW, GW DAH VAKSIN BOO...",positive,"[maju, loe, bible, gak, takut, gw, gw, dah, va...","[maju, bible, takut, vaksin, booster]","{'maju': 1, 'loe': 1, 'bible': 1, 'gak': 1, 't...","[maju, bible, takut, vaksin, booster]","[maju, bible, takut, vaksin, booster]",maju bible takut vaksin booster


# Modeling


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

clf =  RandomForestClassifier(random_state=0, max_features ='sqrt',max_depth = 100)
knn = KNeighborsClassifier(n_neighbors=7)
nb = GaussianNB()
dt = DecisionTreeClassifier(max_features ='sqrt',max_depth = 100)
svm = SVC(degree=3)

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

In [ ]:
tfid = TfidfVectorizer(
    min_df = 3,
    max_df = 0.10,
    ngram_range = (1,2)
)

In [ ]:
count_vect = CountVectorizer(
    min_df = 3,
    max_df = 0.10,
    ngram_range = (1,2)
)

In [ ]:
tfid_result = tfid.fit_transform(df_train["done_text"]).toarray()
tfid_df = pd.DataFrame(tfid_result, columns = tfid.get_feature_names())
tfid_df.columns = ["word_" + str(x) for x in tfid_df.columns]
tfid_df.index = df_train.index
# new_data = pd.concat([, tfid_df], axis=1)

In [ ]:
cvzr_result = count_vect.fit_transform(df_train["done_text"]).toarray()
cvzr_df = pd.DataFrame(cvzr_result, columns = count_vect.get_feature_names())
cvzr_df.columns = ["word_" + str(x) for x in cvzr_df.columns]
cvzr_df.index = df_train.index
# new_data = pd.concat([, tfid_df], axis=1)

In [ ]:
cvzr_df.columns

Index(['word_18', 'word_2x', 'word_abis', 'word_abis vaksin', 'word_asa',
       'word_astra', 'word_ayo', 'word_ayo vaksin', 'word_badan',
       'word_banget', 'word_bekas', 'word_besok', 'word_booster gratis',
       'word_booster pake', 'word_booster prokes', 'word_booster seolaholah',
       'word_booster susah', 'word_booster vaksin', 'word_covid', 'word_demam',
       'word_efek', 'word_efek vaksin', 'word_flight', 'word_gatakut',
       'word_gel', 'word_gin', 'word_gratis', 'word_info', 'word_info vaksin',
       'word_jarak', 'word_kemaren', 'word_lemes', 'word_longgar',
       'word_longgar pakai', 'word_lupa', 'word_lupa prokes', 'word_maju',
       'word_moderna', 'word_pakai', 'word_pakai surat', 'word_pake',
       'word_pake astra', 'word_paksa', 'word_pandemi', 'word_pas',
       'word_peama', 'word_prokes mudik', 'word_prokes vaksin', 'word_ragu',
       'word_ragu vaksin', 'word_rantau', 'word_rasa', 'word_sakit',
       'word_sampe', 'word_sehat', 'word_selamat', 'w

## Menampilkan Hasil classification

In [ ]:
y = df_train.label

In [ ]:
X = cvzr_df.to_numpy()
y = df_train.label
y_pred = cross_val_predict(clf, X, y, cv=5)
conf_mat = confusion_matrix(y_pred, y)

In [ ]:
print(conf_mat)
print(classification_report(y, y_pred, digits = 4))

[[20  4  3]
 [ 7 31 11]
 [ 6  0 22]]
              precision    recall  f1-score   support

    negative     0.7407    0.6061    0.6667        33
      netral     0.6327    0.8857    0.7381        35
    positive     0.7857    0.6111    0.6875        36

    accuracy                         0.7019       104
   macro avg     0.7197    0.7010    0.6974       104
weighted avg     0.7199    0.7019    0.6979       104



In [ ]:
# sns.heatmap(classification_report(y, y_pred, digits = 4), annot =True)

## Menampilkan hasil knn

In [ ]:
print('KNN')
X = cvzr_df.to_numpy()
y = df_train.label
y_pred = cross_val_predict(knn, X, y, cv=5)
conf_mat = confusion_matrix(y_pred, y)
print(conf_mat)
print(classification_report(y, y_pred, digits = 4))

KNN
[[ 5  0  0]
 [27 35 29]
 [ 1  0  7]]
              precision    recall  f1-score   support

    negative     1.0000    0.1515    0.2632        33
      netral     0.3846    1.0000    0.5556        35
    positive     0.8750    0.1944    0.3182        36

    accuracy                         0.4519       104
   macro avg     0.7532    0.4487    0.3790       104
weighted avg     0.7496    0.4519    0.3806       104



In [ ]:
X

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 1],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [ ]:
# sns.heatmap(classification_report(y, y_pred, digits = 4), annot =True)

## Menampilkan hasil Decision Tree

In [ ]:
print('Decision Tree')
X = cvzr_df.to_numpy()
y = df_train.label
y_pred = cross_val_predict(dt, X, y, cv=5)
conf_mat = confusion_matrix(y_pred, y)
print(conf_mat)
print(classification_report(y, y_pred, digits = 4))

Decision Tree
[[21  6  3]
 [ 4 29 12]
 [ 8  0 21]]
              precision    recall  f1-score   support

    negative     0.7000    0.6364    0.6667        33
      netral     0.6444    0.8286    0.7250        35
    positive     0.7241    0.5833    0.6462        36

    accuracy                         0.6827       104
   macro avg     0.6895    0.6828    0.6793       104
weighted avg     0.6897    0.6827    0.6792       104



In [ ]:
# sns.heatmap(classification_report(y, y_pred, digits = 4), annot =True)

# SVM


In [ ]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 104 entries, 0 to 103
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   text                 104 non-null    object
 1   text_real            104 non-null    object
 2   label                104 non-null    object
 3   text_tokens          104 non-null    object
 4   text_tokens_WSW      104 non-null    object
 5   text_tokens_fdist    104 non-null    object
 6   text_normalized      104 non-null    object
 7   text_tokens_stemmed  104 non-null    object
 8   done_text            104 non-null    object
dtypes: object(9)
memory usage: 12.2+ KB


In [ ]:
# def convert(label):
#   if label == 'negative':
#     return 0
#   elif label == "netral":
#     return 1
#   else:
#     return 2

In [ ]:
# df_train['label'] = df_train['label'].apply(convert)

# vactorization

In [ ]:
df_train.head()

,text,text_real,label,text_tokens,text_tokens_WSW,text_tokens_fdist,text_normalized,text_tokens_stemmed,done_text
0,hey coba himbau warganya cek seifikat vaksin d...,Hey @txtdrbogor coba himbau warganya cek serti...,netral,"[hey, coba, himbau, warganya, cek, seifikat, v...","[coba, himbau, warganya, cek, seifikat, vaksin...","{'hey': 1, 'coba': 1, 'himbau': 1, 'warganya':...","[coba, himbau, warganya, cek, seifikat, vaksin...","[coba, himbau, warga, cek, seifikat, vaksin, p...",coba himbau warga cek seifikat vaksin peduli l...
1,mumpung netizen progresif dan berwawasan diban...,RT @Outstandjing: Mumpung netizen progresif da...,netral,"[mumpung, netizen, progresif, dan, berwawasan,...","[mumpung, netizen, progresif, berwawasan, diba...","{'mumpung': 1, 'netizen': 1, 'progresif': 1, '...","[mumpung, netizen, progresif, berwawasan, diba...","[mumpung, netizen, progresif, wawas, banding, ...",mumpung netizen progresif wawas banding keluar...
2,ini setelah vaksin booster jadi batuk mana awe...,Ini setelah vaksin booster jadi batukkkk.. man...,negative,"[ini, setelah, vaksin, booster, jadi, batuk, m...","[vaksin, booster, batuk, awet, sampe, skrng, y...","{'ini': 1, 'setelah': 1, 'vaksin': 1, 'booster...","[vaksin, booster, batuk, awet, sampe, skrng, y...","[vaksin, booster, batuk, awet, sampe, skrng, y...",vaksin booster batuk awet sampe skrng yaallah ...
3,maju loe bible gak takut gw gw dah vaksin booster,"RT @rockmansick: MAJU LOE BIBLE GAK TAKUT GW, ...",positive,"[maju, loe, bible, gak, takut, gw, gw, dah, va...","[maju, bible, takut, vaksin, booster]","{'maju': 1, 'loe': 1, 'bible': 1, 'gak': 1, 't...","[maju, bible, takut, vaksin, booster]","[maju, bible, takut, vaksin, booster]",maju bible takut vaksin booster
4,maju loe bible gak takut gw gw dah vaksin booster,"MAJU LOE BIBLE GAK TAKUT GW, GW DAH VAKSIN BOO...",positive,"[maju, loe, bible, gak, takut, gw, gw, dah, va...","[maju, bible, takut, vaksin, booster]","{'maju': 1, 'loe': 1, 'bible': 1, 'gak': 1, 't...","[maju, bible, takut, vaksin, booster]","[maju, bible, takut, vaksin, booster]",maju bible takut vaksin booster


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
from sklearn.model_selection import train_test_split
# extract the labels from the train data
y = df_train.label.values
# use 70% for the training and 30% for the test
x_train, x_test, y_train, y_test = train_test_split(df_train.done_text.values, y,
stratify=y,
random_state=1,
test_size=0.5, shuffle=True)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
# initializing the countvectorizer
vectorizer = CountVectorizer()
# tokenize and make the document into a matrix
document_term_matrix = vectorizer.fit_transform(df_train['done_text'])
# check the result
df_baru = pd.DataFrame(document_term_matrix.toarray(), columns = vectorizer.get_feature_names())

In [ ]:
stopwords_indo = txt_stopword.stopwords.to_list()

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
# vectorize tweets for model building
vectorizer = CountVectorizer(binary=True, stop_words=stopwords_indo)
# learn a vocabulary dictionary of all tokens in the raw documents
vectorizer.fit(list(x_train) + list(x_test))
# transform documents to document-term matrix
x_train_vec = vectorizer.transform(x_train)
x_test_vec = vectorizer.transform(x_test)

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:401: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['aa', 'aaa', 'aaaahhhh', 'aaaakkkhhhhhh', 'aaah', 'aah', 'aahh', 'aahhh', 'aajh', 'aang', 'ab', 'aba', 'abc', 'abd', 'acan', 'ada', 'adalah', 'adanya', 'adapun', 'ade', 'adoh', 'aduch', 'aduhh', 'ae', 'aeeee', 'aernye', 'aga', 'again', 'agak', 'agaknya', 'agar', 'agree', 'ah', 'ahaaaaa', 'ahahahah', 'ahelah', 'ahh', 'ahhahshahahsha', 'ahhay', 'ahhh', 'ahhhh', 'ahhhhh', 'ahool', 'ai', 'aig', 'aih', 'aiiihhh', 'ailapyupul', 'aing', 'aj', 'aja', 'ajaa', 'ajaaah', 'ajaah', 'ajasi', 'ajib', 'ajj', 'ajjaaa', 'aka', 'akan', 'akankah', 'akankh', 'akh', 'akhir', 'akhiri', 'akhirnya', 'akhirx', 'akse', 'aku', 'akubingin', 'akulah', 'akutuh', 'al', 'alah', 'alangkah', 'alhamdulillah', 'alla', 'alu', 'always', 'am', 'ama', 'amat', 'amatlah', 'amp', 'ampe', 'an', 'anca', 'and', 'anda', 'andalah', 'angg', 

In [ ]:
from sklearn import svm
# classify using support vector classifier
svm = svm.SVC(kernel = 'linear', probability=True)
# fit the SVC model based on the given training data
prob = svm.fit(x_train_vec, y_train).predict_proba(x_test_vec)
# perform classification and prediction on samples in x_test
y_pred_svm = svm.predict(x_test_vec)

In [ ]:
conf_mat = confusion_matrix(y_test, y_pred_svm)
print(conf_mat)
print(classification_report(y_test, y_pred_svm, digits = 4))

[[ 8  9  1]
 [ 0 17  4]
 [ 0 10 34]]
              precision    recall  f1-score   support

    negative     1.0000    0.4444    0.6154        18
      netral     0.4722    0.8095    0.5965        21
    positive     0.8718    0.7727    0.8193        44

    accuracy                         0.7108        83
   macro avg     0.7813    0.6756    0.6771        83
weighted avg     0.7985    0.7108    0.7187        83



In [ ]:
from sklearn.metrics import accuracy_score
print("Accuracy score for SVC is: ", accuracy_score(y_test, y_pred_svm) * 100, '%')

Accuracy score for SVC is:  71.08433734939759 %


In [ ]:
sns.heatmap(conf_mat, annot=True)

In [ ]:
test = vectorizer.transform(['belum vaksin booster dia'])
svm.predict(test)

array(['netral'], dtype=object)

In [ ]:
test = vectorizer.transform(['vaksin booster jangan lupa'])
svm.predict(test)

array(['positive'], dtype=object)

In [ ]:
test = vectorizer.transform(['ga ada efek samping ya sayang ya btw vaksin booster nya itu pzifer'])
svm.predict(test)

array(['netral'], dtype=object)

In [ ]:
test = vectorizer.transform(['nih yaa booster dan prokes adalah dua kunci tak terpisahkan sebab faktanya potensi kenaikan kasus masih tetap ada jika vaksin booster tidak dibarengi dengan disiplin protokol kesehatan'])
svm.predict(test)

array(['positive'], dtype=object)

In [ ]:
test = vectorizer.transform(['bersihin kandang anabul vaksin booster tidur siang gara2 abis vaksin efek ngantuk hype hanabi badan panas dingin gini doang cape lemes jadi satu bye mau tidur lagi'])
svm.predict(test)

array(['negative'], dtype=object)